In [6]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 522.1 kB/s eta 0:00:00m eta 0:00:010:00:02
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [14]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, MeanShift, MiniBatchKMeans, Birch, AffinityPropagation, AgglomerativeClustering
from sklearn import metrics

In [45]:
df_data = pd.read_csv("../../results_demo/T50/FFT/Group_7/fft-Group_7-T50.csv")
df_data

,id,target,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,...,p_246,p_247,p_248,p_249,p_250,p_251,p_252,p_253,p_254,p_255
0,sequence_53082,55.0,5581.3286,1352.9421,263.7332,721.0711,1397.3702,1521.6666,2271.3767,467.4430,...,1469.6219,2201.6253,274.1347,1193.0147,1750.2012,1299.5251,284.9651,2107.7423,576.5233,1726.2032
1,sequence_19402,43.0,4899.0463,1339.3273,1327.5561,265.7643,1275.1395,642.2799,1546.7248,651.5814,...,1355.3607,864.4830,1248.1832,761.0373,2044.5461,1575.8697,739.4454,2053.8490,898.7640,773.2994
2,sequence_20054,49.0,4232.0487,2742.6632,829.8600,507.7918,340.3717,1393.7780,1702.5803,1696.3929,...,1887.9681,1628.1346,969.9843,609.4143,1773.8028,1332.2637,1743.0953,1238.5454,826.2339,530.8567
3,sequence_20048,39.8,4141.1666,1603.1525,491.7341,445.5678,859.0940,1521.1933,1773.0048,963.5555,...,1853.2158,617.8683,873.9271,515.3349,2456.8836,1546.0510,773.3633,685.2298,928.4679,860.0668
4,sequence_20036,52.9,5271.4167,1745.1170,918.7657,490.8281,654.9796,1269.7122,2054.7238,1634.4521,...,1835.9097,1006.0810,944.9125,404.1364,1948.0221,1767.9909,1045.0068,1112.8635,410.3791,416.2101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,sequence_19347,58.9,5706.7836,1261.9006,385.0086,725.2933,1441.0516,1301.9368,2109.3506,517.5597,...,1390.9861,2107.3608,8.6082,1157.6808,1758.7687,1239.7625,412.2078,2378.1352,753.1593,1509.4593
257,sequence_19379,50.3,5970.6152,1062.7149,535.5449,597.1190,1331.6853,1360.8911,2199.2113,551.0165,...,1679.5715,2319.9614,544.0902,956.8960,1604.9936,1003.9962,472.4779,2112.4685,874.6829,1324.0113
258,sequence_19348,46.7,5332.2912,900.9466,654.3819,864.2724,1207.7777,1075.6827,2137.3210,139.5892,...,1338.9243,1386.9814,933.4337,1188.4158,1650.9357,1064.7273,1232.7742,715.3669,1562.2249,659.4333
259,sequence_19380,48.1,5596.1228,706.2604,740.4600,704.3079,1097.4830,1125.3950,2258.8147,225.8817,...,1799.7108,1779.1458,501.8358,1245.9584,1535.1299,647.7608,1063.5335,460.7090,1366.8402,565.4685


In [46]:
def get_performance_clustering(data, labels):
    siluetas = metrics.silhouette_score(data, labels, metric='euclidean')
    calinski = metrics.calinski_harabasz_score(data, labels)
    davies = metrics.davies_bouldin_score(data, labels)

    return siluetas, calinski, davies

In [47]:
#ignoramos y removemos las columnas que no ocuparemos para el clustering
ignore_columns = pd.DataFrame()
ignore_columns['id'] = df_data['id']
ignore_columns['target'] = df_data['target']

df_data = df_data.drop(columns=['id', 'target'])

In [48]:
#exploramos KNN
matrix_result = []
for k in range(2, 30):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_data)
    siluetas, calinski, davies = get_performance_clustering(df_data, kmeans.labels_)
    row = ["k-means-{}".format(k), siluetas, calinski, davies]
    matrix_result.append(row)

df_explore = pd.DataFrame(matrix_result, columns=['strategy', 'siluetas', 'calinski', 'davies'])

In [49]:
#seleccionamos el mejor
import numpy as np

highest_siluetas = np.max(df_explore['siluetas'])
highest_calinski = np.max(df_explore['calinski'])

# filtramos
df_filter_by_siluetas = df_explore.loc[df_explore['siluetas'] >= highest_siluetas]
df_filter_by_calinski = df_explore.loc[df_explore['calinski'] >= highest_calinski]

df_concat = pd.concat([df_filter_by_siluetas, df_filter_by_calinski])
strategies = df_concat['strategy'].unique()
strategies


array(['k-means-2'], dtype=object)

In [50]:
# en este caso salio el mejor k-means=2
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(df_data)
ignore_columns['labels'] = kmeans.labels_

ignore_columns.to_csv("../../results_demo/T50/FFT/Group_7/unsupervised_clustering_sequences.csv")